In [159]:
import numpy as np

import astropy.units as u
from astropy import constants as const
from astropy.cosmology import Planck18 as cosmo
from astropy.modeling.models import BlackBody

import emcee

In [190]:
# Konstanter
h     = const.h.value
c     = const.c.value
k_B   = const.k_B.value
Tcmb0 = cosmo.Tcmb0.value

# Parametrar från sektioner 2 och 3.1 i Bakx et al. (2021)
z          = 7.13
kappa_star = (10.41 * u.cm**2 / u.g).to(u.m**2 / u.kg).value
nu_star    = (1900  * u.GHz).to(u.Hz).value

# Tabell 1 från Bakx et al. (2021)
lambda_obs = ([0.427, 0.728, 0.873, 1.33] *  u.mm).to(u.m ).value
flux_rest  = ([154,   180,   143,   60  ] * u.uJy).to(u.Jy).value
flux_error = ([37,    39,    15,    11  ] * u.uJy).to(u.Jy).value

# Konvertering av våglängderna i tabellen till frekvenser
nu_obs  = c / lambda_obs

# Våglängder och frekvenser i viloram
lambda_rest = lambda_obs / (1 + z)
nu_rest     = nu_obs     * (1 + z)

# Svartkroppsstrålning
B = lambda T, nu: (2 * h * nu**3 / c**2) / (np.exp(h * nu / (k_B * T)) - 1.0)

# Rödförskjutning av bakgrundsstrålning
Tcmb = lambda z: Tcmb0 * (1 + z)

# En konstant som används av ekv (8) i Sommovigo et al. (2021)
g_z = ((1 + z) / (cosmo.luminosity_distance(z)**2)).to(u.m**-2).value

# Konvertera M_sun till kg
M_sun_to_kg = lambda M_sun: (1 * u.M_sun).to(u.kg).value * M_sun

In [193]:
def SED(nu, T_d, beta_d, M_d): 
    # Från ekv (12) i da Cunha et al. (2013)
    T_corrected = (T_d**(4 + beta_d) + Tcmb0**(4 + beta_d) * ((1 + z)**(4 + beta_d) - 1))**(1 / (4 + beta_d))

    # Från ekv (8) i Sommovigo et al. (2021) som redan bakar in ekv (18) i da Cunha et al. (2013)
    kappa_nu = kappa_star * (nu / nu_star) ** beta_d
    flux = g_z * M_sun_to_kg(M_d) * kappa_nu * (B(T_corrected, nu) - B(Tcmb(z), nu))

    # Konvertera (W * m^-2 * Hz^-1) till Jy
    return flux * 1e26

In [171]:
def log_likelihood(params, nu, flux, flux_error):
    T_d, beta_d, M_d = params
    flux_model = SED(nu, T_d, beta_d, M_d)
    residuals = (flux - flux_model) / flux_error
    return -0.5 * np.sum(residuals**2)

def log_prior(params):
    T_d, beta_d, M_d = params
    if 25 < T_d < 80 and 0.2 < beta_d < 2.5 and 1e6 < M_d < 1e12:
        return 0.0
    return -np.inf

def log_probability(params, nu, flux, flux_err):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, nu, flux, flux_err)

In [200]:
n_walkers = 64
n_steps = 10000
pos = [np.random.uniform([30, 0.5, 1e6], [60, 2.5, 1e7]) for _ in range(n_walkers)]

sampler = emcee.EnsembleSampler(n_walkers, 3, log_probability, args=(nu_rest, flux_rest, flux_error))
sampler.run_mcmc(pos, n_steps, progress=True)

flat_samples = sampler.get_chain(discard=2000, thin=15, flat=True)
T_dust_mcmc, beta_mcmc, M_dust_mcmc = np.median(flat_samples, axis=0)

T_dust_err = np.percentile(flat_samples[:, 0], [16, 84])
beta_err = np.percentile(flat_samples[:, 1], [16, 84])
M_dust_err_corr = np.percentile(flat_samples[:, 2], [16, 84])
M_dust_mcmc_corr =  M_dust_mcmc

print(f'T_dust = {T_dust_mcmc:.2f} +{T_dust_err[1]-T_dust_mcmc:.2f}/-{T_dust_mcmc-T_dust_err[0]:.2f} K')
print(f'beta = {beta_mcmc:.2f} +{beta_err[1]-beta_mcmc:.2f}/-{beta_mcmc-beta_err[0]:.2f}')
print(f'M_dust = {M_dust_mcmc_corr:.2e} +{M_dust_err_corr[1]-M_dust_mcmc_corr:.2e}/-{M_dust_mcmc_corr-M_dust_err_corr[0]:.2e} M_sun')

100%|██████████| 10000/10000 [00:25<00:00, 386.71it/s]

T_dust = 42.01 +11.82/-6.96 K
beta = 1.62 +0.59/-0.70
M_dust = 1.66e+07 +1.22e+07/-7.10e+06 M_sun
